In [ ]:
import imagej
import scyjava
import os
import shutil
from datetime import datetime
 
scyjava.config.add_option('-Xmx6g')
#use 'sc.fiji:fiji' otherwise fiji won't be initialized, you can also connect it to a local application (might be better
# in case you have important plugins installed)

ij = imagej.init('C:/fiji-win64/Fiji.app', mode = 'interactive') #This is with local connected Fiji (imageJ) - for more details see: py.imagej.net

def move_images(src_folder, dest_folder, image_extensions=[".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp"]):
            """
            Move all images from the source folder to the destination folder.

            Parameters:
            - src_folder: The source directory containing the images.
            - dest_folder: The destination directory to move the images to.
            - image_extensions: List of file extensions to consider as images. Default includes common image formats.
            """

    
            
            if not os.path.exists(dest_folder):
                os.makedirs(dest_folder)
    
            # Loop through each file in the source folder
            for filename in os.listdir(src_folder):
                if any(filename.endswith(ext) for ext in image_extensions):
                    src_path = os.path.join(src_folder, filename)
                    dest_path = os.path.join(dest_folder, filename)
            
                
                    shutil.move(src_path, dest_path)
                    
                    
 
# Define the directory containing your images
image_directory = #Needs to be defined
image_files = os.listdir(image_directory)

# Define folders for Csv file
source_folder = #Needs to be defined
target_folder = #Needs to be defined

#Give a folder for macro input 
destination_image_path = #Needs to be defined like this: /folder/folder/img.png
 
# Fiji macro
macro = """
// Process the image
open("#insertdirectionhere/img.png") #Give the same direction as 'destination_image_path'
close("dummy")

//Duplicate the original image 
selectWindow("img.png");
run("Duplicate...", "title=original");
selectWindow("img.png");
run("8-bit");
run("Invert");

run("Auto Threshold", "method=Otsu white");

run("Fill Holes");
run("Erode");
run("Watershed");

// Get rid of aggregates
run("Duplicate...", "title=img-1.png");
run("Analyze Particles...", "size=3000-Infinity show=Masks");
run("Invert");
imageCalculator("Subtract create", "Mask of img-1.png","img.png");
selectImage("Result of Mask of img-1.png");
run("Clear Results");
roiManager("reset");
 
//Analyse single cells
run("Invert");
run("Analyze Particles...", "size=200-Infinity circularity=0.50-1.00 show=Nothing display add exclude");
run("Clear Results");

n = roiManager("count");

run("Set Measurements...", "area mean standard modal min centroid center perimeter bounding fit shape feret's integrated median skewness kurtosis area_fraction redirect=None decimal=3");



// Measure ROIs
for (i=0; i<n; i++){
	selectWindow("original");
	roiManager("Select", i); 
	run("Measure");
}


saveAs("Results",'#insertdirectionhere/Measurements.csv'); #Give the same direction as 'source_folder'
roiManager("reset");
run("Close All");
run("Clear Results");
"""


#Analyse all images in the file location
for image_file in image_files: 
    if image_file.endswith(('.tif' , '.jpg' , '.png')):
        
        image_path = os.path.join(image_directory, image_file)
        
        # Get the last modification time
        modification_time = os.path.getmtime(image_path)

        readable_time = datetime.fromtimestamp(modification_time).strftime('%Y-%m-%d %H:%M:%S')
        shutil.copy(image_path, destination_image_path)
        
        
        
    
        ij.py.run_macro('newImage("dummy", "8-bit", 1, 1, 1);')
        
        ij.py.run_macro(macro)
        
        # Rename and move the output CSV file
        original_csv = os.path.join(source_folder, 'Measurements.csv')
        if os.path.exists(original_csv):
            modification_time = os.path.getmtime(image_path)
            readable_time = datetime.fromtimestamp(modification_time).strftime('%Y!%m!%d %H-%M-%S')

            # Generate new file name
            new_csv_name = f'{readable_time}.csv'
            new_csv_path = os.path.join(target_folder, new_csv_name)

            # Rename and move the file
            shutil.move(original_csv, new_csv_path)
            print(f'Renamed and moved CSV to {new_csv_path}')
        
source_folder =  # Replace with your source folder path
target_folder = # Replace with your target folder path

 
# Shutdown ImageJ
ij.getContext().dispose()

## Analyse results (XGBoost)

In [ ]:
import pandas as pd 
import os
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

# Model direction
model_dir = #Needs to be set
model_name = #Needs to be defined
os.chdir(model_dir)

loaded_model = xgb.XGBClassifier()

loaded_model.load_model(model_name)

class_percentages_list = []

# Directory where the CSV files are located
csv_files_dir = #Needs to be defined


# Process each CSV file in the directory
for file_name in os.listdir(csv_files_dir):
    if file_name.endswith('.csv'):
        csv_file_path = os.path.join(csv_files_dir, file_name)

        X_test = pd.read_csv(csv_file_path)

        y_pred = loaded_model.predict(X_test)

        unique, counts = np.unique(y_pred, return_counts=True)
        percentages = dict(zip(unique, counts * 100 / len(y_pred)))

        print(f"Class percentages for {file_name}: {percentages}")

        # Add the file name and percentages to the list
        percentages['file_name'] = file_name
        class_percentages_list.append(percentages)

class_percentages_df = pd.DataFrame(class_percentages_list)

# Set file_name as the first column
cols = ['file_name'] + [col for col in class_percentages_df if col != 'file_name']
class_percentages_df = class_percentages_df[cols]

# Save to CSV file
output_file = #Needs to be defined
class_percentages_df.to_csv(output_file, index=False)

        

